In [1]:
import pandas as pd
import numpy as np
import math
import random
from random import randint
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score, roc_curve, auc

# 划分数据集

In [2]:
def parse_train_test(DataFrame,feature):
    train_xy = pd.read_csv(DataFrame)
    train_xy.drop("Unnamed: 0",axis=1,inplace=True)
    train, val = train_test_split(train_xy, test_size=0.2, random_state=20)
    train_idx = list(train.index)
    val_idx = list(val.index)
    train_y = train[feature]
    train_x = train.drop(feature, axis=1)
    val_y = val[feature]
    val_x = val.drop(feature, axis=1)
    return train_x, train_y, val_x, val_y ,train_idx, val_idx
X_train, y_train, X_test, y_test,train_idx,val_idx = parse_train_test("hob_df.csv","HOB")

# XG筛选变量

In [3]:
def xg_select(X_train, X_test, y_train, y_test,DataFrame,feature):
    df = pd.read_csv(DataFrame)
    xg = xgb.XGBClassifier(use_label_encoder=False)
    xg.fit(X_train, y_train)
    xg_importance = xg.feature_importances_
    xg_importance = xg_importance * 100
    indices = np.argsort(xg_importance)[::-1]
    features = X_train.columns
    xg_feature_list = []
    xg_importance_list = []
    for f in range(X_train.shape[1]):
        print("%2d %-*s %f" % (f + 1, 30, features[indices[f]], xg_importance[indices[f]]))
        xg_feature_list.append(features[indices[f]])
        xg_importance_list.append(xg_importance[indices[f]])
    xg_df = pd.DataFrame(xg_importance_list,xg_feature_list)
    xg_lst = list(xg_df.iloc[:20,:].index)
    xg_lst.append(feature)
    xg_fea = df.loc[:,xg_lst]
    return xg_fea
xg_fea = xg_select(X_train, X_test, y_train, y_test,"hob_df.csv","HOB")
train_x,val_x,train_y,val_y = xg_fea.iloc[train_idx,:-1],xg_fea.iloc[val_idx,:-1],xg_fea.iloc[train_idx,-1],xg_fea.iloc[val_idx,-1]

[20:39:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
 1 nHCsatu                        7.345992
 2 MDEC-23                        4.841626
 3 BCUTc-1l                       4.094265
 4 nBondsM                        2.915586
 5 nHBint3                        2.146558
 6 MLFER_A                        1.688609
 7 nBondsS                        1.474605
 8 SdO                            1.469046
 9 MDEC-11                        1.265488
10 maxdssC                        1.250351
11 SssS                           1.227123
12 SaaO                           1.104204
13 maxHBint6                      1.079011
14 nBondsS2                       1.075755
15 ETA_EtaP_F_L                   1.049051
16 SHssNH                         1.038296
17

In [6]:
def xgb_cla():
    xg = xgb.XGBClassifier(use_label_encoder=False)
    xg.fit(train_x, train_y)
    xg_y = xg.predict(val_x)
    xg_y_prob = xg.predict_proba(val_x)[:,1]
    print("roc_auc_score:%.2f%%"%(metrics.roc_auc_score(val_y,xg_y) * 100.0))
    print("accuracy_score:%.2f%%"%(accuracy_score(val_y,xg_y) * 100.0))
    print("recall_score:%.2f%%"%(recall_score(val_y,xg_y) * 100.0))
    print("f1_score:%.2f%%"%(f1_score(val_y,xg_y) * 100.0))
    print("precision_score:%.2f%%"%(precision_score(val_y,xg_y) * 100.0))
    
    return xg_y,xg_y_prob

In [7]:
xg_y,xg_y_prob = xgb_cla()

[20:39:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
roc_auc_score:81.33%
accuracy_score:85.87%
recall_score:72.53%
f1_score:71.35%
precision_score:70.21%


# GA-XG

In [8]:
generations = 200   # 繁殖代数 200
pop_size = 100      # 种群数量  100
max_value = 10     # 基因中允许出现的最大值
chrom_length = 15  # 染色体长度  
pc = 0.6           # 交配概率  
pm = 0.01          # 变异概率  
results = []       # 存储每一代的最优解，N个三元组（auc最高值, n_estimators, max_depth）  
fit_value = []     # 个体适应度  
fit_mean = []      # 平均适应度 
random_seed = 20
cons_value = 0.19 / 31 # (0.20-0.01）/ (32 - 1)



dtrain = xgb.DMatrix(train_x, label=train_y)
dval = xgb.DMatrix(val_x)


def xgboostModel(tree_num, eta, max_depth, min_child_weight,random_seed):
    
    params = {
        'booster': 'gbtree',
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'eta': eta,  # 0.02
        'max_depth': max_depth, # 8
        'min_child_weight': min_child_weight, # 3
        'gamma': 0.1,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'lambda': 550,
        'alpha': 19,
        'seed': randint(1,10000),
        'nthread': 3
    }
    model = xgb.train(params, dtrain, num_boost_round=tree_num)
    predict_y = model.predict(dval, iteration_range = (0,0))
    roc_auc = metrics.roc_auc_score(val_y, predict_y)
    return roc_auc 

# Step1:对参数进行编码
def geneEncoding(pop_size, chrom_length):  
    pop = [[]]
    for i in range(pop_size):
        temp = []
        for j in range(chrom_length):
            temp.append(random.randint(0, 1))
        pop.append(temp)
    return pop[1:]

# Step 2:计算个体的目标函数值
def cal_obj_value(pop):
    objvalue = []
    variable = decodechrom(pop)
    for i in range(len(variable)):
        tempVar = variable[i]
 
        tree_num_value = (tempVar[0] + 1)* 10
        eta_value = 0.01 + tempVar[1] * cons_value
        max_depth_value = 3 + tempVar[2]
        min_child_weight_value = 1 + tempVar[3]
 
        aucValue = xgboostModel(tree_num_value, eta_value, max_depth_value, min_child_weight_value, random_seed)
        objvalue.append(aucValue)
    return objvalue 
 
# 对每个个体进行解码
def decodechrom(pop):
    variable = []
    for i in range(len(pop)):
        res = []
        
        # 计算第一个变量值，即 0101->10(逆转)
        temp1 = pop[i][0:4]
        v1 = 0
        for i1 in range(4):
            v1 += temp1[i1] * (math.pow(2, i1))
        res.append(int(v1))
        
        # 计算第二个变量值
        temp2 = pop[i][4:9]
        v2 = 0
        for i2 in range(5):
            v2 += temp2[i2] * (math.pow(2, i2))
        res.append(int(v2))
 
        # 计算第三个变量值
        temp3 = pop[i][9:12]
        v3 = 0
        for i3 in range(3):
            v3 += temp3[i3] * (math.pow(2, i3))
        res.append(int(v3))
 
        # 计算第四个变量值
        temp4 = pop[i][12:15]
        v4 = 0
        for i4 in range(3):
            v4 += temp4[i4] * (math.pow(2, i4))
        res.append(int(v4))
 
        variable.append(res)
    return variable            # 500*4

# Step 3:计算个体的适应值，即最大值，淘汰负值
def calfitvalue(obj_value):
    fit_value = []
    temp = 0.0
    Cmin = 0
    for i in range(len(obj_value)):
        if(obj_value[i] + Cmin > 0):
            temp = Cmin + obj_value[i]
        else:
            temp = 0.0
        fit_value.append(temp)
    return fit_value
 
# Step 4:找出适应函数值中最大值,和对应的个体
def best(pop, fit_value):
    best_individual = pop[0]
    best_fit = fit_value[0]
    for i in range(1, len(pop)):
        if(fit_value[i] > best_fit):
            best_fit = fit_value[i]
            best_individual = pop[i]
    return [best_individual, best_fit]
 
# Step 5:每次繁殖，将最好的结果记录下来(将二进制转化为十进制)
def b2d(best_individual):
    # 计算第一个变量值
    temp1 = best_individual[0:4]
    v1 = 0
    for i1 in range(4):
        v1 += temp1[i1] * (math.pow(2, i1))
    v1 = (v1 + 1) * 10
    
    # 计算第二个变量值
    temp2 = best_individual[4:9]
    v2 = 0
    for i2 in range(5):
        v2 += temp2[i2] * (math.pow(2, i2))
    v2 = 0.01 + v2 * cons_value
 
    # 计算第三个变量值
    temp3 = best_individual[9:12]
    v3 = 0
    for i3 in range(3):
        v3 += temp3[i3] * (math.pow(2, i3))
    v3 = 3 + v3
 
    # 计算第四个变量值
    temp4 = best_individual[12:15]
    v4 = 0
    for i4 in range(3):
        v4 += temp4[i4] * (math.pow(2, i4))
    v4 = 1 + v4
 
    return int(v1), float(v2), int(v3), int(v4)

# Step 6:自然选择（轮盘赌算法）
def selection(pop, fit_value):
    # 计算每个适应值的概率
    new_fit_value = []
    total_fit = sum(fit_value)
    for i in range(len(fit_value)):
        new_fit_value.append(fit_value[i] / total_fit)
        
    # 计算每个适应值的累积概率
    cumsum(new_fit_value)
    
    # 生成随机浮点数序列
    ms = []
    pop_len = len(pop)
    for i in range(pop_len):
        ms.append(random.random())
    ms.sort()
    # 轮盘赌算法
    fitin = 0
    newin = 0
    newpop = pop
    while newin < pop_len:
        if(ms[newin] < new_fit_value[fitin]):
            newpop[newin] = pop[fitin]
            newin = newin + 1
        else:
            fitin = fitin + 1
    pop = newpop

    
# 求适应值的总和
def sum(fit_value):
    total = 0
    for i in range(len(fit_value)):
        total += fit_value[i]
    return total
 
# 计算累积概率
def cumsum(fit_value):
    temp=[]
    for i in range(len(fit_value)):
        t = 0
        j = 0
        while(j <= i):
            t += fit_value[j]
            j = j + 1
        temp.append(t)
    for i in range(len(fit_value)):
        fit_value[i]=temp[i]

        
# Step 7:交叉繁殖
def crossover(pop, pc): #个体间交叉，实现基因交换
    poplen = len(pop)
    for i in range(poplen - 1):
        if(random.random() < pc):
            cpoint = random.randint(0,len(pop[0]))
            temp1 = []
            temp2 = []
            temp1.extend(pop[i][0 : cpoint])
            temp1.extend(pop[i+1][cpoint : len(pop[i])])
            temp2.extend(pop[i+1][0 : cpoint])
            temp2.extend(pop[i][cpoint : len(pop[i])])
            pop[i] = temp1
            pop[i+1] = temp2
 
 
# Step 8:基因突变
def mutation(pop, pm):
    px = len(pop)
    py = len(pop[0])
    for i in range(px):
        if(random.random() < pm):
            mpoint = random.randint(0, py-1)
            if(pop[i][mpoint] == 1):
                pop[i][mpoint] = 0
            else:
                pop[i][mpoint] = 1


def generAlgo(generations):
    pop = geneEncoding(pop_size, chrom_length)
    print(str(generations)+" start...")
    for i in range(generations):
        obj_value = cal_obj_value(pop) # 计算目标函数值
        fit_value = calfitvalue(obj_value) #计算个体的适应值
        [best_individual, best_fit] = best(pop, fit_value) #选出最好的个体和最好的函数值
        v1, v2, v3, v4 = b2d(best_individual)
        results.append([best_fit, v1, v2, v3, v4]) #每次繁殖，将最好的结果记录下来
        selection(pop, fit_value) #自然选择，淘汰掉一部分适应性低的个体
        crossover(pop, pc) #交叉繁殖
        mutation(pop, pc) #基因突变
    results.sort()
    print(results[-1])
    best_v1,best_v2,best_v3,best_v4 = results[-1][1],results[-1][2],results[-1][3],results[-1][4]
    print(best_v1,best_v2,best_v3,best_v4)
    return best_v1,best_v2,best_v3,best_v4

best_v1,best_v2,best_v3,best_v4,= generAlgo(300)

100 start...
[0.9154349171954805, 90, 0.1570967741935484, 8, 7]
90 0.1570967741935484 8 7


In [9]:
def gaxg_cla():
    gaxg = xgb.XGBClassifier(n_estimators = best_v1, eta = best_v2, max_depth = best_v3, 
                                  min_child_weight = best_v4,objective="multi:softmax",
                                  num_class=2,use_label_encoder=False)
    
    gaxg.fit(train_x, train_y)
    gaxg_y = gaxg.predict(val_x)
    gaxg_y_prob = gaxg.predict_proba(val_x)[:,1]
    
    print("roc_auc_score:%.2f%%"%(metrics.roc_auc_score(val_y,gaxg_y) * 100.0))
    print("accuracy_score:%.2f%%"%(accuracy_score(val_y,gaxg_y) * 100.0))
    print("recall_score:%.2f%%"%(recall_score(val_y,gaxg_y) * 100.0))
    print("f1_score:%.2f%%"%(f1_score(val_y,gaxg_y) * 100.0))
    print("precision_score:%.2f%%"%(precision_score(val_y,gaxg_y) * 100.0))
    return gaxg_y,gaxg_y_prob
gaxg_y,gaxg_y_prob = gaxg_cla()


[20:53:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softmax' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
roc_auc_score:82.59%
accuracy_score:87.20%
recall_score:73.63%
f1_score:73.63%
precision_score:73.63%


# RF 筛选变量

In [10]:
def rf_select(X_train, X_test, y_train, y_test,DataFrame,feature):
    df = pd.read_csv(DataFrame)
    rfc = RandomForestClassifier()
    rfc.fit(X_train, y_train)
    rfc_importance = rfc.feature_importances_
    rfc_importance = rfc_importance * 100
    indices_ = np.argsort(rfc_importance)[::-1]
    features = X_train.columns
    rfc_feature_list = []
    rfc_importance_list = []
    for f in range(X_train.shape[1]):
        print("%2d %-*s %f" % (f + 1, 30, features[indices_[f]], rfc_importance[indices_[f]]))
        rfc_feature_list.append(features[indices_[f]])
        rfc_importance_list.append(rfc_importance[indices_[f]])
    rfc_df = pd.DataFrame(rfc_importance_list,rfc_feature_list)
    rfc_lst = list(rfc_df.iloc[:20,:].index)
    rfc_lst.append(feature)
    rfc_fea = df.loc[:,rfc_lst]
    return rfc_fea
rfc_fea = rf_select(X_train, X_test, y_train, y_test,"hob_df.csv","HOB")
train_x,val_x,train_y,val_y = rfc_fea.iloc[train_idx,:-1],rfc_fea.iloc[val_idx,:-1],rfc_fea.iloc[train_idx,-1],rfc_fea.iloc[val_idx,-1]

 1 BCUTc-1l                       3.934259
 2 SsOH                           2.815611
 3 nsOH                           1.941140
 4 maxsOH                         1.795923
 5 SHsOH                          1.575139
 6 maxHsOH                        1.486456
 7 nHsOH                          1.258028
 8 MLFER_A                        1.079127
 9 minHsOH                        0.981461
10 minsOH                         0.976974
11 minaasC                        0.969008
12 VP-3                           0.852605
13 maxHBd                         0.835856
14 nHBDon                         0.707867
15 VP-5                           0.695908
16 BCUTp-1l                       0.670317
17 maxssCH2                       0.669373
18 maxdO                          0.662375
19 VP-7                           0.649613
20 SdO                            0.642188
21 SHBd                           0.615810
22 VP-6                           0.608692
23 nHBAcc                         0.597719
24 hmin    

# RF

In [12]:
def rf_cla():
    rfc = RandomForestClassifier()
    rfc.fit(train_x, train_y)
    rf_y = rfc.predict(val_x)
    rf_y_prob = rfc.predict_proba(val_x)[:,1]
    
    print("roc_auc_score:%.2f%%"%(metrics.roc_auc_score(val_y,rf_y) * 100.0))
    print("accuracy_score:%.2f%%"%(accuracy_score(val_y,rf_y) * 100.0))
    print("recall_score:%.2f%%"%(recall_score(val_y,rf_y) * 100.0))
    print("f1_score:%.2f%%"%(f1_score(val_y,rf_y) * 100.0))
    print("precision_score:%.2f%%"%(precision_score(val_y,rf_y) * 100.0))
    return rf_y,rf_y_prob
rf_y,rf_y_prob = rf_cla()

roc_auc_score:85.49%
accuracy_score:89.33%
recall_score:78.02%
f1_score:78.02%
precision_score:78.02%


In [13]:
generations = 300   # 繁殖代数 300
pop_size = 100       # 种群数量  100
max_value = 10      # 基因中允许出现的最大值  
chrom_length = 16   # 染色体长度  
pc = 0.6            # 交配概率  
pm = 0.01           # 变异概率  
results = [[]]      # 存储每一代的最优解，N个三元组（auc最高值, n_estimators, max_depth）  
fit_value = []      # 个体适应度  
fit_mean = []       # 平均适应度 
pop = [[0, 1, 0, 1, 0, 1, 0, 1] for i in range(pop_size)] # 初始化种群中所有个体的基因初始序列
 
 
 
 
'''
n_estimators 取 {10、20、30、40、50、60、70、80、90、100、110、120、130、140、150、160}
max_depth 取 {1、2、3、4、5、6、7、8、9、10、11、12、13、14、15、16} 
（1111，1111）基因组8位长
'''
def randomForest(n_estimators_value, max_depth_value):
 
 
    # print("n_estimators_value: " + str(n_estimators_value))
    # print("max_depth_value: " + str(max_depth_value))
 
    rf = RandomForestClassifier(n_estimators=n_estimators_value,
                                max_depth=max_depth_value,
                                n_jobs=2)
    rf.fit(train_x, train_y)  # 训练分类器
    predict_test = rf.predict_proba(val_x)[:, 1]
    roc_auc = metrics.roc_auc_score(val_y, predict_test)
    # auc = metrics.auc(val_y,predict_test)
    return roc_auc
 

# Step 1 : 对参数进行编码（用于初始化基因序列，可以选择初始化基因序列，本函数省略）
def geneEncoding(pop_size, chrom_length):  
    pop = [[]]
    for i in range(pop_size):
        temp = []
        for j in range(chrom_length):
            temp.append(random.randint(0, 1))
        pop.append(temp)
    return pop[1:]
 
# Step 2 : 计算个体的目标函数值
def cal_obj_value(pop):
    objvalue = []
    variable = decodechrom(pop)
    for i in range(len(variable)):
        tempVar = variable[i]
        n_estimators_value = (tempVar[0] + 1) * 10
        max_depth_value = tempVar[1] + 1
        aucValue = randomForest(n_estimators_value, max_depth_value)
        objvalue.append(aucValue)
    return objvalue #目标函数值objvalue[m] 与个体基因 pop[m] 对应 
 
 
 
# 对每个个体进行解码，并拆分成单个变量，返回 n_estimators 和 max_depth
def decodechrom(pop):
    variable = []
    n_estimators_value = []
    max_depth_value = []
    for i in range(len(pop)):
        res = []
        
        # 计算第一个变量值，即 0101->10(逆转)
        temp1 = pop[i][0:4]
        preValue = 0
        for pre in range(4):
            preValue += temp1[pre] * (math.pow(2, pre))
        res.append(int(preValue))
        
        # 计算第二个变量值
        temp2 = pop[i][4:8]
        aftValue = 0
        for aft in range(4):
            aftValue += temp2[aft] * (math.pow(2, aft))
        res.append(int(aftValue))
        variable.append(res)
    return variable
 
 
 
# Step 3: 计算个体的适应值（计算最大值，于是就淘汰负值就好了）
def calfitvalue(obj_value):
    fit_value = []
    temp = 0.0
    Cmin = 0
    for i in range(len(obj_value)):
        if(obj_value[i] + Cmin > 0):
            temp = Cmin + obj_value[i]
        else:
            temp = 0.0
        fit_value.append(temp)
    return fit_value
 
 
 
# Step 4: 找出适应函数值中最大值，和对应的个体
def best(pop, fit_value):
    best_individual = pop[0]
    best_fit = fit_value[0]
    for i in range(1, len(pop)):
        if(fit_value[i] > best_fit):
            best_fit = fit_value[i]
            best_individual = pop[i]
    return [best_individual, best_fit]

# Step 5: 每次繁殖，将最好的结果记录下来(将二进制转化为十进制)
def b2d(best_individual):
    temp1 = best_individual[0:4]
    preValue = 0
    for pre in range(4):
        preValue += temp1[pre] * (math.pow(2, pre))
    preValue = preValue + 1
    preValue = preValue * 10
    
    # 计算第二个变量值
    temp2 = best_individual[4:8]
    aftValue = 0
    for aft in range(4):
        aftValue += temp2[aft] * (math.pow(2, aft))
    aftValue = aftValue + 1
    return int(preValue), int(aftValue)
 
 
 
# Step 6: 自然选择（轮盘赌算法）
def selection(pop, fit_value):
    # 计算每个适应值的概率
    new_fit_value = []
    total_fit = sum(fit_value)
    for i in range(len(fit_value)):
        new_fit_value.append(fit_value[i] / total_fit)
    # 计算每个适应值的累积概率
    cumsum(new_fit_value)
    # 生成随机浮点数序列
    ms = []
    pop_len = len(pop)
    for i in range(pop_len):
        ms.append(random.random())
    # 对生成的随机浮点数序列进行排序
    ms.sort()
    # 轮盘赌算法（选中的个体成为下一轮，没有被选中的直接淘汰，被选中的个体代替）
    fitin = 0
    newin = 0
    newpop = pop
    while newin < pop_len:
        if(ms[newin] < new_fit_value[fitin]):
            newpop[newin] = pop[fitin]
            newin = newin + 1
        else:
            fitin = fitin + 1
    pop = newpop
 
 
# 求适应值的总和
def sum(fit_value):
    total = 0
    for i in range(len(fit_value)):
        total += fit_value[i]
    return total
 
# 计算累积概率
def cumsum(fit_value):
    temp=[]
    for i in range(len(fit_value)):
        t = 0
        j = 0
        while(j <= i):
            t += fit_value[j]
            j = j + 1
        temp.append(t)
    for i in range(len(fit_value)):
        fit_value[i]=temp[i]
 
 
# Step 7: 交叉繁殖
def crossover(pop, pc): #个体间交叉，实现基因交换
    poplen = len(pop)
    for i in range(poplen - 1):
        if(random.random() < pc):
            cpoint = random.randint(0,len(pop[0]))
            temp1 = []
            temp2 = []
            temp1.extend(pop[i][0 : cpoint])
            temp1.extend(pop[i+1][cpoint : len(pop[i])])
            temp2.extend(pop[i+1][0 : cpoint])
            temp2.extend(pop[i][cpoint : len(pop[i])])
            pop[i] = temp1
            pop[i+1] = temp2
 
 
 
 
# Step 8: 基因突变
def mutation(pop, pm):
    px = len(pop)
    py = len(pop[0])
    for i in range(px):
        if(random.random() < pm):
            mpoint = random.randint(0,py-1)
            if(pop[i][mpoint] == 1):
                pop[i][mpoint] = 0
            else:
                pop[i][mpoint] = 1
                
for i in range(100):
    print("第 " + str(i) + " 代开始繁殖......")
    obj_value = cal_obj_value(pop) # 计算目标函数值
    # print(obj_value)
    fit_value = calfitvalue(obj_value) #计算个体的适应值
    # print(fit_value)
    [best_individual, best_fit] = best(pop, fit_value) #选出最好的个体和最好的函数值
    # print("best_individual: "+ str(best_individual))
    temp_n_estimator, temp_max_depth = b2d(best_individual)
    results.append([best_fit, temp_n_estimator, temp_max_depth]) #每次繁殖，将最好的结果记录下来
    print(str(best_individual) + " " + str(best_fit))
    selection(pop, fit_value) #自然选择，淘汰掉一部分适应性低的个体
    crossover(pop, pc) #交叉繁殖
    mutation(pop, pc) #基因突变
results.sort()
print(results[-1])
n_estimators_value,max_depth_value = results[-1][1],results[-1][2]

第 0 代开始繁殖......
[0, 1, 0, 1, 0, 1, 0, 1] 0.948846927720167
第 1 代开始繁殖......
[0, 1, 0, 1, 0, 1, 1, 1] 0.949698189134809
第 2 代开始繁殖......
[0, 1, 0, 1, 1, 1, 0, 1] 0.9491951710261569
第 3 代开始繁殖......
[0, 1, 0, 0, 0, 1, 1, 1] 0.9474346076458753
第 4 代开始繁殖......
[0, 1, 0, 1, 1, 0, 1, 1] 0.9477248104008668
第 5 代开始繁殖......
[0, 1, 0, 1, 1, 1, 0, 1] 0.9469896300882217
第 6 代开始繁殖......
[1, 1, 0, 1, 1, 0, 0, 1] 0.9470670174895527
第 7 代开始繁殖......
[1, 1, 0, 0, 1, 1, 0, 1] 0.9466026930815663
第 8 代开始繁殖......
[0, 0, 0, 1, 1, 1, 0, 1] 0.9466800804828974
第 9 代开始繁殖......
[1, 1, 0, 1, 1, 1, 0, 1] 0.9482278285095187
第 10 代开始繁殖......
[1, 1, 0, 1, 1, 1, 1, 1] 0.9471057111902184
第 11 代开始繁殖......
[1, 1, 1, 0, 1, 0, 0, 1] 0.9474539544962081
第 12 代开始繁殖......
[1, 1, 0, 0, 1, 1, 0, 1] 0.9489823556724966
第 13 代开始繁殖......
[1, 1, 0, 1, 1, 1, 0, 1] 0.9486534592168394
第 14 代开始繁殖......
[1, 1, 0, 0, 1, 1, 0, 1] 0.9488275808698345
第 15 代开始繁殖......
[1, 1, 1, 1, 1, 1, 0, 1] 0.9484212970128463
第 16 代开始繁殖......
[1, 1, 1, 1, 1, 1, 

# GA-RF

In [20]:
def garf_cla():
    garf = RandomForestClassifier(n_estimators = n_estimators_value, max_depth = max_depth_value)
    garf.fit(train_x,train_y)
    garf_y = garf.predict(val_x)
    garf_y_prob = garf.predict_proba(val_x)[:,1]
    
    print("roc_auc_score:%.2f%%"%(metrics.roc_auc_score(val_y,garf_y) * 100.0))
    print("accuracy_score:%.2f%%"%(accuracy_score(val_y,garf_y) * 100.0))
    print("recall_score:%.2f%%"%(recall_score(val_y,garf_y) * 100.0))
    print("f1_score:%.2f%%"%(f1_score(val_y,garf_y) * 100.0))
    print("precision_score:%.2f%%"%(precision_score(val_y,garf_y) * 100.0))
    return garf_y,garf_y_prob
garf_y,garf_y_prob = garf_cla()

roc_auc_score:84.76%
accuracy_score:88.80%
recall_score:76.92%
f1_score:76.92%
precision_score:76.92%


# GA-XG-RF

In [22]:
DNA_SIZE = 24
POP_SIZE = 100
CROSSOVER_RATE = 0.8
MUTATION_RATE = 0.005
N_GENERATIONS = 100
W_XG_BOUND = [0, 1]
W_RF_BOUND = [0, 1]
test_y = np.array(val_y)


def F(W_XG):
    
    y_t = test_y
    xg = gaxg_y
    rf = garf_y
    W_XG = W_XG.reshape(100,1)
    W_RF = (1-W_XG).reshape(100,1)
    entropy_loss = -np.sum(y_t * np.log(W_XG * xg + W_RF * rf+0.000001),axis=1)
    return entropy_loss.reshape(100,1)

def get_fitness(pop): 
    W_XG,W_RF = translateDNA(pop)
    pred = F(W_XG)
    fitness = -(pred - np.max(pred)) + 1e-3
    return  fitness.reshape(100,)#减去最小的适应度是为了防止适应度出现负数，通过这一步fitness的范围为[0, np.max(pred)-np.min(pred)],最后在加上一个很小的数防止出现为0的适应度


def translateDNA(pop): #pop表示种群矩阵，一行表示一个二进制编码表示的DNA，矩阵的行数为种群数目
    W_XG_pop = pop[:,1::2]#奇数列表示X
    W_RF_pop = pop[:,::2] #偶数列表示y

    #pop:(POP_SIZE,DNA_SIZE)*(DNA_SIZE,1) --> (POP_SIZE,1)
    W_XG = W_XG_pop.dot(2**np.arange(DNA_SIZE)[::-1])/float(2**DNA_SIZE-1)*(W_XG_BOUND[1]-W_XG_BOUND[0])+W_XG_BOUND[0]
    W_RF = W_RF_pop.dot(2**np.arange(DNA_SIZE)[::-1])/float(2**DNA_SIZE-1)*(W_RF_BOUND[1]-W_RF_BOUND[0])+W_RF_BOUND[0]
    return W_XG,W_RF

def crossover_and_mutation(pop, CROSSOVER_RATE = 0.8):
    new_pop = []
    for father in pop:		#遍历种群中的每一个个体，将该个体作为父亲
        child = father		#孩子先得到父亲的全部基因（这里我把一串二进制串的那些0，1称为基因）
        if np.random.rand() < CROSSOVER_RATE:            #产生子代时不是必然发生交叉，而是以一定的概率发生交叉
            mother = pop[np.random.randint(POP_SIZE)]    #再种群中选择另一个个体，并将该个体作为母亲
            cross_points = np.random.randint(low=0, high=DNA_SIZE*2)  #随机产生交叉的点
            child[cross_points:] = mother[cross_points:] #孩子得到位于交叉点后的母亲的基因
        mutation(child)	#每个后代有一定的机率发生变异
        new_pop.append(child)

    return new_pop

def mutation(child, MUTATION_RATE=0.03):
    if np.random.rand() < MUTATION_RATE:                 #以MUTATION_RATE的概率进行变异
        mutate_point = np.random.randint(0, DNA_SIZE*2)  #随机产生一个实数，代表要变异基因的位置
        child[mutate_point] = child[mutate_point]^1 #    将变异点的二进制为反转

def select(pop, fitness):    # nature selection wrt pop's fitness
    idx = np.random.choice(np.arange(POP_SIZE), size=POP_SIZE, replace=True,
                           p=(fitness)/(fitness.sum()) )
    return pop[idx]

def print_info(pop):
    fitness = get_fitness(pop)
    max_fitness_index = np.argmax(fitness)

    print("max_fitness:", fitness[max_fitness_index])
    W_XG,W_RF = translateDNA(pop)
    print("最优的基因型：", pop[max_fitness_index])
    print("(W_XG),(W_RF):", (W_XG[max_fitness_index]),(1-W_XG[max_fitness_index]))
    return ( W_XG[max_fitness_index],1-W_XG[max_fitness_index])



pop = np.random.randint(2, size=(POP_SIZE, DNA_SIZE*2)) #matrix (POP_SIZE, DNA_SIZE)
for _ in range(N_GENERATIONS):#迭代N代
    W_XG,W_RF = translateDNA(pop)
    pop = np.array(crossover_and_mutation(pop, CROSSOVER_RATE))
    fitness = get_fitness(pop)
    pop = select(pop, fitness) #选择生成新的种群
print_info(pop)
fitness = get_fitness(pop)
max_fitness_index = np.argmax(fitness)
W_XG,W_RF = translateDNA(pop)
xg_weight,rf_weight = W_XG[max_fitness_index],1-W_XG[max_fitness_index]

max_fitness: 0.0010598490262859741
最优的基因型： [0 0 0 1 1 1 1 0 0 1 1 0 1 1 1 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 1 0 1 0 1
 1 1 1 0 0 0 1 1 1 0 1]
(W_XG),(W_RF): 0.41408106172567977 0.5859189382743202


In [23]:
gaxgrf_y = xg_weight * xg_y_prob+ rf_weight *  rf_y_prob

fpr_xg, tpr_xg, thred_xg = roc_curve(y_test,xg_y_prob,pos_label=1)
fpr_gaxg, tpr_gaxg, thred_gaxg = roc_curve(y_test,gaxg_y_prob,pos_label=1)
fpr_rf, tpr_rf, thred_rf = roc_curve(val_y,rf_y_prob,pos_label=1)
fpr_garf, tpr_garf, thred_garf = roc_curve(val_y,garf_y_prob,pos_label=1)
fpr_gaxgrf, tpr_gaxgrf, thred_gaxgrf = roc_curve(val_y,gaxgrf_y,pos_label=1)

label = ['XGBoost','GA-XGBoost', 'RF','GA-RF','GA-XGRF']
plt.plot(fpr_xg,tpr_xg,"--")
plt.plot(fpr_gaxg,tpr_gaxg,"-")
plt.plot(fpr_rf,tpr_rf,"-.")
plt.plot(fpr_garf,tpr_garf,":")
plt.plot(fpr_gaxgrf,tpr_gaxgrf,".-")

plt.plot([0,1],[0,1],'d--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(label, loc = 4)
plt.title('HOB ROC Curve')
plt.savefig("ROC_4.png",dpi = 300)

array([4.00135978e-05, 7.64849342e-01, 2.12759318e-04, 9.59547850e-01,
       4.25163824e-01, 5.56654091e-01, 6.01987616e-02, 7.86505524e-01,
       9.65429524e-01, 9.67969350e-02, 2.78411037e-03, 3.06319743e-01,
       4.34136329e-02, 2.50652130e-03, 9.24144653e-01, 9.58518902e-01,
       1.87270428e-02, 6.45389345e-01, 6.64265792e-03, 6.74795274e-01,
       1.13129521e-04, 6.03269299e-02, 8.73964928e-01, 3.15633323e-03,
       6.55477144e-01, 7.30508698e-01, 7.27516890e-05, 9.79042816e-05,
       1.19209983e-02, 1.17515136e-02, 2.56737904e-03, 1.76700160e-02,
       3.14448931e-04, 2.41981140e-02, 4.49475673e-02, 5.46441064e-02,
       6.25237948e-03, 7.92658302e-01, 4.44051519e-04, 6.83493970e-04,
       3.54112576e-02, 1.34955134e-02, 5.47636941e-04, 8.78281484e-04,
       6.23145292e-02, 5.76207451e-01, 1.32564121e-02, 3.95380812e-05,
       3.91553061e-01, 5.30060629e-01, 5.14277388e-02, 9.87305362e-01,
       3.39647086e-05, 1.17510750e-02, 1.33347669e-02, 3.34547251e-04,
      